# On The Free Energy Penalty of Cavity Formation In Salt Solutions: Rethinking the Terms “Kosmotropic” and “Chaotropic”.

Stefan Hervø-Hansen<sup>a,*</sup> and Nobuyuki Matubayasi<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## Simulations

### Introduction

### Methods & Materials

### References



## Import of Python Modules & Auxiliary Functions

In [ ]:
# Notebook dependent libs
import numpy as np
import matplotlib.pyplot as plt
import os, time
from shutil import copyfile
from distutils.spawn import find_executable
import parmed as pmd
import mdtraj as md

# Check for external programs
if None in [find_executable('packmol')]:
    print('WARNING: External program missing!')

# homedir = !pwd
# homedir = homedir[0]

homedir = '/home/lin/data/Cavity-Formation'
mddir = '/home/lin/data/Cavity-Formation/md'
print(homedir)

## Molecular Dynamics Simulations
### Simulation settings

In [ ]:
salts = {
    # 'NaF': {'cation': 'Na', 'anion': 'F'},
    'NaCl': {'cation': 'Na', 'anion': 'Cl'},
    # 'NaI': {'cation': 'Na', 'anion': 'I'}
}

concentrations = {
#     0.0:{'N_water':10000,'N_cation':0,'N_anion':0},
    # 0.25:{'N_water':10000,'N_cation':45,'N_anion':45},
    0.5:{'N_water':10000,'N_cation':90,'N_anion':90},
#     0.75:{'N_water':10000,'N_cation':135,'N_anion':135},
    # 1.00:{'N_water':10000,'N_cation':180,'N_anion':180}
}

#### Generation of initial configuration using Packmol

In [ ]:
packmol_script="""
tolerance 2.0
filetype pdb
output system.pdb

add_box_sides 1.0

structure ../../../../PDB_files/OPC.pdb
        number {N_water}
        inside cube 0. 0. 0. 63.
end structure

structure ../../../../PDB_files/Ions/{anion}.pdb
        number {N_anion}
        inside cube 0. 0. 0. 63.
end structure

structure ../../../../PDB_files/Ions/{cation}.pdb
        number {N_cation}
        inside cube 0. 0. 0. 63.
end structure
"""

topology_script = """
; topol.top
[ defaults ]
; nbfunc comb-rule gen-pairs fudgeLJ fudgeQQ
1 2 yes 0.5 0.8333

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
Na            11  22.989769  0.00000000  A        0.25658      0.0971555
Ag            47 107.868220  0.00000000  A        0.26531      0.1492163
Br            35  79.904100  0.00000000  A        0.41195      2.7048107
Cl            17  35.453200  0.00000000  A        0.38309      2.1820915
Cs            55 132.905452  0.00000000  A        0.35458      1.5913927
Cu            29  63.546300  0.00000000  A        0.21399      0.0046986
F              9  18.998403  0.00000000  A        0.30647      0.6360531
I             53 126.904473  0.00000000  A        0.45846      3.3280127
K             19  39.098310  0.00000000  A        0.30968      0.6903724
Li             3   6.941200  0.00000000  A        0.23252      0.0218984
Rb            37  85.467830  0.00000000  A        0.32108      0.8985523
Tl            81 204.383320  0.00000000  A        0.32874       1.049263
OW            OW      0.0000  0.0000  A   3.16655e-01  8.903586e-01
HW            HW      0.0000  0.0000  A   0.00000e+00  0.00000e+00
MW            MW      0.0000  0.0000  A   0.00000e+00  0.00000e+00

#include "../../../../Force_fields/OPC.itp"
#include "../../../../Force_fields/ions.itp"

[ system ]
{cation}{anion} in water

[ molecules ]
; Compound nmols
HOH 10000
{cation} {N_cation}
{anion} {N_anion}
"""

In [ ]:
# for concentration,N in concentrations.items():
#     print(N['N_cation'])

%cd /home/lin/data/Cavity-Formation/Simulations

homedir = '/home/lin'
mddir = '/home/lin/data/Cavity-Formation/md'
outputdir = '/home/lin/data/Cavity-Formation/Simulations'

for salt,S in salts.items():
    for concentration,N in concentrations.items():
        # making directory
        %mkdir {mddir}/{salt}
        %mkdir {outputdir}/{salt}
        %mkdir {mddir}/{salt}/{concentration} 
        %mkdir {outputdir}/{salt}/{concentration}
        folder_names = ['sys','em','nvt','npt']
        for folder_name in folder_names:
            %mkdir {mddir}/{salt}/{concentration}/{folder_name}
#             working_dirs_key = f'{mddir}/{salt}/{concentration}/{folder_name}'
#             working_dirs[f'{salt}_{concentration}'] = f'{mddir}/{salt}/{concentration}/{folder_name}' ＃上書きされるので後で修正
        
        %cd {mddir}/{salt}/{concentration}/sys
        # Generate initial configuration using packmol
        with open('packmol.inp', 'w') as f:
            f.write(packmol_script.format(homedir=homedir,cation=S['cation'], anion=S['anion'],N_water=N['N_water'],
                                          N_anion=N['N_anion'],N_cation=N['N_cation']))
        f.close()
        !packmol < packmol.inp

        # Convert packmol pdb to gromacs format
        init = pmd.load_file('system.pdb')
        init.save('system.gro')

        # Write gromacs topology
        with open('topol.top', 'w') as f:
            f.write(topology_script.format(cation=S['cation'], anion=S['anion'],N_water=N['N_water'],
                                          N_anion=N['N_anion'],N_cation=N['N_cation']))
        f.close()
        # print(working_dirs)
        # print('Wrote initial configurations to'+os.getcwd())

#### Submit script

In [ ]:
md_script = """#!/bin/bash
#PBS -l nodes=1:nu-g:ppn=18

if [ "$PBS_O_WORKDIR" != "" ];then
  cd $PBS_O_WORKDIR
fi
NUMNODES=`wc -l < $PBS_NODEFILE`

source /data2/APPLI/packages/gromacs/gromacs-2021.4-intel-gpu.rc

cd {mddir}/{salt}/{concentration}

#em

cd em

gmx_mpi grompp -f ../../../../MDP_files/em.mdp -c ../sys/system.gro -p ../sys/topol.top -o solution_min.tpr -maxwarn 1
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_min -gpu_id {gpu_id}

cd ..

#NVT

cd nvt

gmx_mpi grompp -f ../../../../MDP_files/nvt.mdp -c ../em/solution_min.gro -p ../sys/topol.top -o solution_nvt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_nvt -gpu_id {gpu_id}

cd ..

#NPT

cd npt

gmx_mpi grompp -f ../../../../MDP_files/npt.mdp -c ../nvt/solution_nvt.gro -p ../sys/topol.top -o solution_npt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_npt -x -gpu_id {gpu_id}

cd ../

#md

cd md

gmx_mpi grompp -f ../../../../MDP_files/md.mdp -c ../npt/solution_npt.gro -p ../sys/topol.top -o solution_run.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm {cation}{anion}_run -x -s solution_run.tpr -gpu_id {gpu_id}

cd ../..

"""

In [ ]:
%cd /home/lin/data/Cavity-Formation/Simulations

homedir = '/home/lin'
mddir = '/home/lin/data/Cavity-Formation/md'
outputdir = '/home/lin/data/Cavity-Formation/Simulations'

tag = 0
for concentration,N in concentrations.items():
    for salt,S in salts.items():
        if tag % 2 == 0:
            gpu_id = 0
        else:
            gpu_id = 1
        #mdを行う
        %cd {mddir}/{salt}/{concentration} 
        with open('run.sh', 'w') as f:
            f.write(md_script.format(gpu_id=gpu_id,mddir=mddir,salt=salt,concentration=concentration,cation=S['cation'],anion=S['anion']))
        f.close()
        print(salt)
        # !qsub run.sh
        tag = tag+1

# RESEARCH NOTES
_will be deleted when the study is finished_

In [ ]:
numbers_to_output = [1,2,3,4]
second_numbers = [1,2]
i = 0
for number in numbers_to_output:
    for second_number in second_numbers:
        print ('begin loop')
        if i % 2 == 0:
            print ('first number is')
            print (number)
            print ('second number is')
            print (second_number)
        i = i+1
        print ('end loop')